In [1]:
import optuna
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder
from xgboost import XGBClassifier

In [2]:
# Read dataset
df = pd.read_csv("predictive_maintenance.csv").drop(['Target'], axis=1).dropna().drop_duplicates()

In [3]:
# Apply LabelEncoder to target column
df['Failure Type'] = LabelEncoder().fit_transform(df['Failure Type'])

# Apply OrdinalEncoder to categorical features
categorical_features = df.select_dtypes(include='object').columns
df[categorical_features] = OrdinalEncoder().fit_transform(df[categorical_features])

# Apply MinMaxScaler to all continous features
continuous_features = df.select_dtypes(include=['float64', 'int64']).columns
df[continuous_features] = MinMaxScaler().fit_transform(df[continuous_features])

In [4]:
# Remove special characters from column names
df.columns = df.columns.str.replace('[^\w\s]', '', regex=True)

In [5]:
# Split dataset into train and test subsets
X = df.drop(['Failure Type'], axis=1)
y = df['Failure Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
# Perform hyperparameter optimization
def objective(trial):
    try:
        eta = trial.suggest_float('eta', 0.01, 0.3, step=0.01)
        max_depth = trial.suggest_int('max_depth', 3, 10)
        subsample = trial.suggest_float('subsample', 0.5, 1.0, step=0.1)
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.1)
        min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
        gamma = trial.suggest_float('gamma', 0.0, 1.0, step=0.1)
        reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0, step=0.1)
        reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0, step=0.1)
        clf = XGBClassifier(
            objective='multi:softmax',
            eval_metric='mlogloss',
            eta=eta,
            max_depth=max_depth,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            min_child_weight=min_child_weight,
            gamma=gamma,
            reg_lambda=reg_lambda,
            reg_alpha=reg_alpha,
            n_jobs=-1,
            random_state=42
        )
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        trial.set_user_attr('classification_report', report)
        return accuracy
    except:
        raise optuna.TrialPruned()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000, n_jobs=-1)

[I 2023-06-02 14:02:58,808] A new study created in memory with name: no-name-605f5122-0079-4c50-862a-acb7c9c58936
[I 2023-06-02 14:03:00,384] Trial 15 finished with value: 0.9836666666666667 and parameters: {'eta': 0.26, 'max_depth': 3, 'subsample': 0.7, 'colsample_bytree': 0.5, 'min_child_weight': 7, 'gamma': 1.0, 'reg_lambda': 1.0, 'reg_alpha': 0.2}. Best is trial 15 with value: 0.9836666666666667.
[I 2023-06-02 14:03:00,435] Trial 0 finished with value: 0.9836666666666667 and parameters: {'eta': 0.27, 'max_depth': 6, 'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 10, 'gamma': 0.0, 'reg_lambda': 0.6000000000000001, 'reg_alpha': 0.30000000000000004}. Best is trial 15 with value: 0.9836666666666667.
[I 2023-06-02 14:03:00,510] Trial 13 finished with value: 0.9836666666666667 and parameters: {'eta': 0.17, 'max_depth': 9, 'subsample': 0.5, 'colsample_bytree': 0.8, 'min_child_weight': 8, 'gamma': 0.4, 'reg_lambda': 0.6000000000000001, 'reg_alpha': 0.6000000000000001}. Best

[I 2023-06-02 14:03:03,736] Trial 27 finished with value: 0.9833333333333333 and parameters: {'eta': 0.22, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.6, 'min_child_weight': 10, 'gamma': 0.0, 'reg_lambda': 0.4, 'reg_alpha': 0.30000000000000004}. Best is trial 9 with value: 0.9843333333333333.
[I 2023-06-02 14:03:04,045] Trial 26 finished with value: 0.9836666666666667 and parameters: {'eta': 0.3, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 2, 'gamma': 0.0, 'reg_lambda': 0.8, 'reg_alpha': 0.30000000000000004}. Best is trial 9 with value: 0.9843333333333333.
[I 2023-06-02 14:03:04,950] Trial 30 finished with value: 0.983 and parameters: {'eta': 0.22, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 1.0, 'min_child_weight': 5, 'gamma': 0.2, 'reg_lambda': 0.0, 'reg_alpha': 0.0}. Best is trial 9 with value: 0.9843333333333333.
[I 2023-06-02 14:03:05,156] Trial 31 finished with value: 0.9833333333333333 and parameters: {'eta': 0.22, 'max_de

[I 2023-06-02 14:03:08,620] Trial 53 finished with value: 0.9833333333333333 and parameters: {'eta': 0.25, 'max_depth': 9, 'subsample': 0.9, 'colsample_bytree': 0.8, 'min_child_weight': 6, 'gamma': 0.1, 'reg_lambda': 0.1, 'reg_alpha': 0.9}. Best is trial 9 with value: 0.9843333333333333.
[I 2023-06-02 14:03:08,689] Trial 54 finished with value: 0.9823333333333333 and parameters: {'eta': 0.25, 'max_depth': 9, 'subsample': 0.9, 'colsample_bytree': 0.8, 'min_child_weight': 6, 'gamma': 0.1, 'reg_lambda': 0.5, 'reg_alpha': 0.9}. Best is trial 9 with value: 0.9843333333333333.
[I 2023-06-02 14:03:08,914] Trial 58 finished with value: 0.983 and parameters: {'eta': 0.26, 'max_depth': 6, 'subsample': 0.6, 'colsample_bytree': 0.8, 'min_child_weight': 8, 'gamma': 0.1, 'reg_lambda': 0.5, 'reg_alpha': 0.8}. Best is trial 9 with value: 0.9843333333333333.
[I 2023-06-02 14:03:08,983] Trial 56 finished with value: 0.9826666666666667 and parameters: {'eta': 0.25, 'max_depth': 9, 'subsample': 0.9, 'cols

[I 2023-06-02 14:03:14,658] Trial 78 finished with value: 0.9826666666666667 and parameters: {'eta': 0.09, 'max_depth': 7, 'subsample': 1.0, 'colsample_bytree': 0.9, 'min_child_weight': 4, 'gamma': 0.7000000000000001, 'reg_lambda': 0.2, 'reg_alpha': 1.0}. Best is trial 65 with value: 0.985.
[I 2023-06-02 14:03:14,747] Trial 83 finished with value: 0.9843333333333333 and parameters: {'eta': 0.12, 'max_depth': 10, 'subsample': 1.0, 'colsample_bytree': 0.9, 'min_child_weight': 7, 'gamma': 0.4, 'reg_lambda': 0.2, 'reg_alpha': 0.8}. Best is trial 65 with value: 0.985.
[I 2023-06-02 14:03:14,757] Trial 81 finished with value: 0.9836666666666667 and parameters: {'eta': 0.11, 'max_depth': 10, 'subsample': 1.0, 'colsample_bytree': 0.9, 'min_child_weight': 7, 'gamma': 0.4, 'reg_lambda': 0.2, 'reg_alpha': 0.8}. Best is trial 65 with value: 0.985.
[I 2023-06-02 14:03:14,873] Trial 82 finished with value: 0.9833333333333333 and parameters: {'eta': 0.09999999999999999, 'max_depth': 10, 'subsample': 

[I 2023-06-02 14:03:19,647] Trial 109 finished with value: 0.9833333333333333 and parameters: {'eta': 0.13, 'max_depth': 9, 'subsample': 0.7, 'colsample_bytree': 0.8, 'min_child_weight': 3, 'gamma': 0.4, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.6000000000000001}. Best is trial 104 with value: 0.9853333333333333.
[I 2023-06-02 14:03:19,948] Trial 110 finished with value: 0.9856666666666667 and parameters: {'eta': 0.18000000000000002, 'max_depth': 9, 'subsample': 0.7, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.0, 'reg_lambda': 0.4, 'reg_alpha': 0.7000000000000001}. Best is trial 110 with value: 0.9856666666666667.
[I 2023-06-02 14:03:20,119] Trial 106 finished with value: 0.9843333333333333 and parameters: {'eta': 0.13, 'max_depth': 10, 'subsample': 0.9, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.5, 'reg_lambda': 0.8, 'reg_alpha': 0.7000000000000001}. Best is trial 110 with value: 0.9856666666666667.
[I 2023-06-02 14:03:20,440] Trial 107 finished w

...

[I 2023-06-02 14:05:40,287] Trial 871 finished with value: 0.987 and parameters: {'eta': 0.13, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.1, 'reg_lambda': 0.1, 'reg_alpha': 0.1}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:05:40,701] Trial 873 finished with value: 0.987 and parameters: {'eta': 0.13, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.1, 'reg_lambda': 0.1, 'reg_alpha': 0.1}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:05:40,808] Trial 872 finished with value: 0.9863333333333333 and parameters: {'eta': 0.13, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.1, 'reg_lambda': 0.1, 'reg_alpha': 0.0}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:05:41,054] Trial 874 finished with value: 0.9863333333333333 and parameters: {'eta': 0.13, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2,

[I 2023-06-02 14:05:46,024] Trial 898 finished with value: 0.986 and parameters: {'eta': 0.14, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.1, 'reg_lambda': 0.1, 'reg_alpha': 0.1}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:05:46,053] Trial 900 finished with value: 0.986 and parameters: {'eta': 0.14, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.1, 'reg_lambda': 0.1, 'reg_alpha': 0.1}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:05:46,175] Trial 901 finished with value: 0.986 and parameters: {'eta': 0.14, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.1, 'reg_lambda': 0.1, 'reg_alpha': 0.1}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:05:46,278] Trial 902 finished with value: 0.986 and parameters: {'eta': 0.14, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.1, 'reg_lambda

[I 2023-06-02 14:05:51,628] Trial 928 finished with value: 0.986 and parameters: {'eta': 0.12, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.1, 'reg_alpha': 0.1}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:05:51,910] Trial 929 finished with value: 0.9866666666666667 and parameters: {'eta': 0.12, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.1, 'reg_lambda': 0.1, 'reg_alpha': 0.1}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:05:52,483] Trial 930 finished with value: 0.9823333333333333 and parameters: {'eta': 0.13, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.9, 'reg_lambda': 0.1, 'reg_alpha': 0.8}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:05:52,590] Trial 931 finished with value: 0.9856666666666667 and parameters: {'eta': 0.13, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_chil

[I 2023-06-02 14:05:57,834] Trial 957 finished with value: 0.986 and parameters: {'eta': 0.13, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.2, 'reg_lambda': 0.1, 'reg_alpha': 0.1}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:05:58,035] Trial 959 finished with value: 0.986 and parameters: {'eta': 0.13, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.2, 'reg_lambda': 0.1, 'reg_alpha': 0.1}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:05:58,103] Trial 958 finished with value: 0.9863333333333333 and parameters: {'eta': 0.11, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.2, 'reg_lambda': 0.1, 'reg_alpha': 0.1}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:05:58,191] Trial 960 finished with value: 0.9863333333333333 and parameters: {'eta': 0.11, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2,

[I 2023-06-02 14:06:03,468] Trial 986 finished with value: 0.9856666666666667 and parameters: {'eta': 0.14, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.1, 'reg_lambda': 0.0, 'reg_alpha': 0.1}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:06:03,670] Trial 988 finished with value: 0.9856666666666667 and parameters: {'eta': 0.14, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.1, 'reg_lambda': 0.0, 'reg_alpha': 0.1}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:06:03,690] Trial 987 finished with value: 0.9856666666666667 and parameters: {'eta': 0.14, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.1, 'reg_lambda': 0.0, 'reg_alpha': 0.1}. Best is trial 638 with value: 0.987.
[I 2023-06-02 14:06:03,738] Trial 989 finished with value: 0.9856666666666667 and parameters: {'eta': 0.14, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0

In [7]:
# Print best trial
print(f"\nBest trial: {study.best_trial.params} => {study.best_trial.values[0]:.3f}")
print(f"\n{pd.DataFrame.from_dict(study.best_trial.user_attrs['classification_report']).round(3).T}")


Best trial: {'eta': 0.13, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 2, 'gamma': 0.1, 'reg_lambda': 0.1, 'reg_alpha': 0.1} => 0.987

              precision  recall  f1-score   support
0                 0.786   0.957     0.863    23.000
1                 0.990   0.998     0.994  2903.000
2                 0.900   0.818     0.857    22.000
3                 0.923   0.774     0.842    31.000
4                 0.000   0.000     0.000     7.000
5                 0.000   0.000     0.000    14.000
accuracy          0.987   0.987     0.987     0.987
macro avg         0.600   0.591     0.593  3000.000
weighted avg      0.980   0.987     0.983  3000.000
